# 공공데이터 포털 동네예보 API를 활용해 내일 기상 예보 받아보기

공공데이터 포털 동네예보 API 바로가기 => https://data.go.kr/data/15057682/openapi.do

API는 신청하여 인증키를 발급 받은 후 사용 가능합니다.

자세한 사용법은 해당 페이지 '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'를 통해 확인 가능합니다.

In [1]:
import pandas as pd
import urllib
import urllib.request
import json

API 사용 신청후 발급받은 일반 인증키(Decoding)를 ServiceKey.txt에 저장하여 불러왔습니다.

인증키 확인 페이지 바로가기(신청 후 사용 가능) => https://data.go.kr/iim/api/selectAPIAcountView.do

In [2]:
ServiceKey = open('API_key/ServiceKey.txt', 'r').read()

In [3]:
ServiceKey

'BBRcDz5+Ggw2/8yIMzcvvHJqWT4tDjkklzSz0GtgEkfby5SrFrCC9hkyo2y0bpEWjEKySRbU5WEA7Oo4meg07Q=='

In [4]:
base_date = '20210613'
fcst_date = '2021-06-14'

In [5]:
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst'

queryParams = '?' + urllib.parse.urlencode(
    {
        urllib.parse.quote_plus('ServiceKey') : ServiceKey, # key를 바로 입력해도 됩니다.
        urllib.parse.quote_plus('numOfRows') : '113', # 총 14개의 항목을 3시간 단위로 순차적으로 불러옵니다. 다음날 24시간예보에 필요한 만큼만 가져왔습니다.
        urllib.parse.quote_plus('dataType') : 'JSON', # JSON, XML 두가지 포멧을 제공합니다.
        urllib.parse.quote_plus('base_date') : base_date, # 예보 받을 날짜를 입력합니다. 최근 1일간의 자료만 제공합니다.
        urllib.parse.quote_plus('base_time') : '1700', # 예보 시간을 입력. 2시부터 3시간 단위
        urllib.parse.quote_plus('nx') : '102', # 울산 태양광 발전소 x 좌표
        urllib.parse.quote_plus('ny') : '83' # 울산 태양광 발전소 y 좌표
    }
)
# 35 114 석문면
response = urllib.request.urlopen(url + queryParams).read()
response = json.loads(response)

In [6]:
fcst_df = pd.DataFrame()
date = fcst_date
fcst_df['Forecast_time'] = [f'{date} {hour}:00' for hour in range(24)]
row_idx = 0

for i, data in enumerate(response['response']['body']['items']['item']):
    if i > 10:
        if data['category']=='REH':
            fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
            print('category:Humidity,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='T3H':
            fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
            print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='SKY':
            fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
            print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='VEC':
            fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
            print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='WSD':
            fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
            print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
            row_idx+=3

category:Humidity, REH baseTime: 1700 , fcstTime: 0000 , fcstValue: 85
category:Cloud, SKY baseTime: 1700 , fcstTime: 0000 , fcstValue: 4
category:Temperature, T3H baseTime: 1700 , fcstTime: 0000 , fcstValue: 22
category:WindDirection, VEC baseTime: 1700 , fcstTime: 0000 , fcstValue: 180
category:WindSpeed, WSD baseTime: 1700 , fcstTime: 0000 , fcstValue: 0.5 

category:Humidity, REH baseTime: 1700 , fcstTime: 0300 , fcstValue: 90
category:Cloud, SKY baseTime: 1700 , fcstTime: 0300 , fcstValue: 4
category:Temperature, T3H baseTime: 1700 , fcstTime: 0300 , fcstValue: 22
category:WindDirection, VEC baseTime: 1700 , fcstTime: 0300 , fcstValue: 34
category:WindSpeed, WSD baseTime: 1700 , fcstTime: 0300 , fcstValue: 0.7 

category:Humidity, REH baseTime: 1700 , fcstTime: 0600 , fcstValue: 90
category:Cloud, SKY baseTime: 1700 , fcstTime: 0600 , fcstValue: 4
category:Temperature, T3H baseTime: 1700 , fcstTime: 0600 , fcstValue: 22
category:WindDirection, VEC baseTime: 1700 , fcstTime: 0600 ,

예보를 한시간 단위로 선형보간합니다.

In [7]:
fcst_df = fcst_df.interpolate()
fcst_df = fcst_df.iloc[:24]
fcst_df

,Forecast_time,Humidity,Cloud,Temperature,WindDirection,WindSpeed
0,2021-06-14 0:00,85.000000,4.0,22.000000,180.000000,0.500000
1,2021-06-14 1:00,86.666667,4.0,22.000000,131.333333,0.566667
2,2021-06-14 2:00,88.333333,4.0,22.000000,82.666667,0.633333
3,2021-06-14 3:00,90.000000,4.0,22.000000,34.000000,0.700000
4,2021-06-14 4:00,90.000000,4.0,22.000000,36.000000,1.000000
5,2021-06-14 5:00,90.000000,4.0,22.000000,38.000000,1.300000
6,2021-06-14 6:00,90.000000,4.0,22.000000,40.000000,1.600000
7,2021-06-14 7:00,86.666667,4.0,22.333333,39.666667,1.766667
8,2021-06-14 8:00,83.333333,4.0,22.666667,39.333333,1.933333
9,2021-06-14 9:00,80.000000,4.0,23.000000,39.000000,2.100000


In [8]:
fcst_df.to_csv(f"private_test/ulsan_fcst_{fcst_date}.csv", index=False)

In [10]:
queryParams = '?' + urllib.parse.urlencode(
    {
        urllib.parse.quote_plus('ServiceKey') : ServiceKey, # key를 바로 입력해도 됩니다.
        urllib.parse.quote_plus('numOfRows') : '113', # 총 14개의 항목을 3시간 단위로 순차적으로 불러옵니다. 다음날 24시간예보에 필요한 만큼만 가져왔습니다.
        urllib.parse.quote_plus('dataType') : 'JSON', # JSON, XML 두가지 포멧을 제공합니다.
        urllib.parse.quote_plus('base_date') : base_date, # 예보 받을 날짜를 입력합니다. 최근 1일간의 자료만 제공합니다.
        urllib.parse.quote_plus('base_time') : '1700', # 예보 시간을 입력. 2시부터 3시간 단위
        urllib.parse.quote_plus('nx') : '35', # 당진 태양광 발전소 x 좌표
        urllib.parse.quote_plus('ny') : '114' # 당진 태양광 발전소 y 좌표
    }
)
response = urllib.request.urlopen(url + queryParams).read()
response = json.loads(response)

In [11]:
fcst_df = pd.DataFrame()
date = fcst_date
fcst_df['Forecast_time'] = [f'{date} {hour}:00' for hour in range(24)]
row_idx = 0

for i, data in enumerate(response['response']['body']['items']['item']):
    if i > 10:
        if data['category']=='REH':
            fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
            print('category:Humidity,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='T3H':
            fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
            print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='SKY':
            fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
            print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='VEC':
            fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
            print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='WSD':
            fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
            print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
            row_idx+=3

category:Humidity, REH baseTime: 1700 , fcstTime: 0000 , fcstValue: 85
category:Cloud, SKY baseTime: 1700 , fcstTime: 0000 , fcstValue: 1
category:Temperature, T3H baseTime: 1700 , fcstTime: 0000 , fcstValue: 19
category:WindDirection, VEC baseTime: 1700 , fcstTime: 0000 , fcstValue: 276
category:WindSpeed, WSD baseTime: 1700 , fcstTime: 0000 , fcstValue: 1.8 

category:Humidity, REH baseTime: 1700 , fcstTime: 0300 , fcstValue: 90
category:Cloud, SKY baseTime: 1700 , fcstTime: 0300 , fcstValue: 1
category:Temperature, T3H baseTime: 1700 , fcstTime: 0300 , fcstValue: 18
category:WindDirection, VEC baseTime: 1700 , fcstTime: 0300 , fcstValue: 315
category:WindSpeed, WSD baseTime: 1700 , fcstTime: 0300 , fcstValue: 2.1 

category:Humidity, REH baseTime: 1700 , fcstTime: 0600 , fcstValue: 85
category:Cloud, SKY baseTime: 1700 , fcstTime: 0600 , fcstValue: 1
category:Temperature, T3H baseTime: 1700 , fcstTime: 0600 , fcstValue: 18
category:WindDirection, VEC baseTime: 1700 , fcstTime: 0600 

In [12]:
fcst_df = fcst_df.interpolate()
fcst_df = fcst_df.iloc[:24]
fcst_df

,Forecast_time,Humidity,Cloud,Temperature,WindDirection,WindSpeed
0,2021-06-14 0:00,85.000000,1.000000,19.000000,276.000000,1.800000
1,2021-06-14 1:00,86.666667,1.000000,18.666667,289.000000,1.900000
2,2021-06-14 2:00,88.333333,1.000000,18.333333,302.000000,2.000000
3,2021-06-14 3:00,90.000000,1.000000,18.000000,315.000000,2.100000
4,2021-06-14 4:00,88.333333,1.000000,18.000000,220.333333,1.600000
5,2021-06-14 5:00,86.666667,1.000000,18.000000,125.666667,1.100000
6,2021-06-14 6:00,85.000000,1.000000,18.000000,31.000000,0.600000
7,2021-06-14 7:00,85.000000,1.666667,18.000000,78.000000,1.133333
8,2021-06-14 8:00,85.000000,2.333333,18.000000,125.000000,1.666667
9,2021-06-14 9:00,85.000000,3.000000,18.000000,172.000000,2.200000


In [13]:
fcst_df.to_csv(f"private_test/dangjin_fcst_{fcst_date}.csv", index=False)